In [1]:
import os
import re
import numpy as np
import pandas as pd

In [28]:
df = pd.read_csv(r'D:\VDownload\NYC Yellow Taxi data\green_tripdata_2013-08_2.csv',
                     sep=r'(?:,|\t|\r|\f|\v)',
                    engine='python')

In [29]:
df.head()

VendorID lpep_pickup_datetime  \
2 2013-08-01 08:14:37  2013-08-01 09:09:06                    N   
  2013-08-01 09:13:00  2013-08-01 11:38:00                    N   
  2013-08-01 09:48:00  2013-08-01 09:49:00                    N   
  2013-08-01 10:38:35  2013-08-01 10:38:51                    N   
  2013-08-01 11:51:45  2013-08-01 12:03:52                    N   

                       Lpep_dropoff_datetime  Store_and_fwd_flag  RateCodeID  \
2 2013-08-01 08:14:37                      1                 0.0         0.0   
  2013-08-01 09:13:00                      1                 0.0         0.0   
  2013-08-01 09:48:00                      5                 0.0         0.0   
  2013-08-01 10:38:35                      1                 0.0         0.0   
  2013-08-01 11:51:45                      1                 0.0         0.0   

                       Pickup_longitude  Pickup_latitude  Dropoff_longitude  \
2 2013-08-01 08:14:37               0.0              0.0                  1   
  2013-08-01 09:13:00               0.0              0.0                  2   
  2013-08-01 09:48:00               0.0              0.0                  1   
  2013-08-01 10:38:35               0.0              0.0                  1   
  2013-08-01 11:51:45               0.0              0.0                  1   

                       Dropoff_latitude  Passenger_count  Trip_distance  \
2 2013-08-01 08:14:37               0.0            21.25            0.0   
  2013-08-01 09:13:00               0.0            74.50            0.0   
  2013-08-01 09:48:00               0.0             1.00            0.1   
  2013-08-01 10:38:35               0.0             3.25            0.0   
  2013-08-01 11:51:45               0.0             8.50            0.0   

                       Fare_amount  Extra  MTA_tax  Tip_amount  Tolls_amount  \
2 2013-08-01 08:14:37          0.0    0.0      0.0         NaN         21.25   
  2013-08-01 09:13:00          0.5    0.0      0.0         NaN         75.00   
  2013-08-01 09:48:00          0.0    0.0      1.0         NaN          2.10   
  2013-08-01 10:38:35          0.0    0.0      0.0         NaN          3.25   
  2013-08-01 11:51:45          0.5    0.0      0.0         NaN          9.00   

                       Ehail_fee  Total_amount  Payment_type  Trip_type  
2 2013-08-01 08:14:37          2           NaN           NaN        NaN  
  2013-08-01 09:13:00          2           NaN           NaN        NaN  
  2013-08-01 09:48:00          2           NaN           NaN        NaN  
  2013-08-01 10:38:35          2           NaN           NaN        NaN  
  2013-08-01 11:51:45          2           NaN           NaN        NaN

In [33]:
reader = pd.read_csv(r'D:\VDownload\NYC Yellow Taxi data\green_tripdata_2013-08_2.csv',
                     sep=',',nrows=0)
#                      chunksize=10)

In [36]:
reader.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 20 columns):
VendorID                 0 non-null object
lpep_pickup_datetime     0 non-null object
Lpep_dropoff_datetime    0 non-null object
Store_and_fwd_flag       0 non-null object
RateCodeID               0 non-null object
Pickup_longitude         0 non-null object
Pickup_latitude          0 non-null object
Dropoff_longitude        0 non-null object
Dropoff_latitude         0 non-null object
Passenger_count          0 non-null object
Trip_distance            0 non-null object
Fare_amount              0 non-null object
Extra                    0 non-null object
MTA_tax                  0 non-null object
Tip_amount               0 non-null object
Tolls_amount             0 non-null object
Ehail_fee                0 non-null object
Total_amount             0 non-null object
Payment_type             0 non-null object
Trip_type                0 non-null object
dtypes: object(20)
memory usage: 0.0+ bytes


In [37]:
reader = pd.read_csv(r'D:\VDownload\NYC Yellow Taxi data\yellow_tripdata_2012-03.csv',
                     sep=',',
                     nrows=0)

In [38]:
reader.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 18 columns):
vendor_id             0 non-null object
pickup_datetime       0 non-null object
dropoff_datetime      0 non-null object
passenger_count       0 non-null object
trip_distance         0 non-null object
pickup_longitude      0 non-null object
pickup_latitude       0 non-null object
rate_code             0 non-null object
store_and_fwd_flag    0 non-null object
dropoff_longitude     0 non-null object
dropoff_latitude      0 non-null object
payment_type          0 non-null object
fare_amount           0 non-null object
surcharge             0 non-null object
mta_tax               0 non-null object
tip_amount            0 non-null object
tolls_amount          0 non-null object
total_amount          0 non-null object
dtypes: object(18)
memory usage: 0.0+ bytes


In [ ]:
reader.info()

In [ ]:
reader

In [12]:
for chunk in reader:
    print(chunk.shape[0])
#     print(chunk.head())
    print('='*50)

KeyboardInterrupt: 

In [14]:
n = 1

In [ ]:
while n:
    

In [15]:
l = [1,2,3]

In [16]:
print(reader.next())

AttributeError: 'TextFileReader' object has no attribute 'next'

In [ ]:
import re

In [ ]:
m = re.match(r'a([bcd]*)e','abce')

In [ ]:
m.string

In [ ]:
sep=r'(?:,|\s+)'